# 通过构建流域mask计算流域平均

## 生成流域mask

In [1]:
import geopandas as gpd
from mean import gen_mask

In [2]:
watershed = gpd.read_file("./shp/337-d8flowdir.shp")
watershed

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/envs/gis-base/share/proj failed


,ORDER,STAID,LON,LAT,AREA,geometry
0,0,Anonymity,127.191667,49.650833,185.512913,"POLYGON ((126.98708 49.80625, 126.98792 49.806..."
1,1,1-1,127.562500,49.398333,197.122011,"POLYGON ((127.54208 49.40042, 127.54292 49.400..."
2,2,2-2,130.145000,48.397500,271.712120,"POLYGON ((129.93042 48.46125, 129.93125 48.461..."
3,3,3-3,132.775000,45.298333,332.742823,"POLYGON ((132.71625 45.53125, 132.71708 45.531..."
4,4,4-4,130.439167,44.680833,3495.662071,"POLYGON ((130.28708 44.75625, 130.28792 44.756..."
...,...,...,...,...,...,...
332,247,247-247,123.690000,46.790000,162762.902859,"POLYGON ((125.11042 51.66042, 125.11625 51.660..."
333,248,248-248,123.857500,46.297500,164072.146744,"POLYGON ((125.11042 51.66042, 125.11625 51.660..."
334,249,249-249,124.266667,45.547500,234370.313479,"POLYGON ((125.11042 51.66042, 125.11625 51.660..."
335,288,288-288,127.242500,45.986667,434704.479349,"POLYGON ((125.11042 51.66042, 125.11625 51.660..."


In [7]:
gen_mask(watershed, "STAID", "gfs", save_dir="./mask")

## 读取流域mask

In [1]:
import xarray as xr

In [2]:
mask = xr.open_dataset("mask/mask-226-226-era5.nc")
mask

<xarray.Dataset>
Dimensions:  (lon: 115, lat: 101)
Coordinates:
  * lon      (lon) float64 119.4 119.5 119.6 119.7 ... 130.5 130.6 130.7 130.8
  * lat      (lat) float64 41.7 41.8 41.9 42.0 42.1 ... 51.3 51.4 51.5 51.6 51.7
Data variables:
    w        (lon, lat) float64 ...

## 获取era5

In [3]:
from wis_s3api import era5
import numpy as np

In [10]:
start_time = np.datetime64('2018-01-01T00:00:00.000000000')
end_time = np.datetime64('2021-12-31T00:00:00.000000000')

In [11]:
bbox = (mask.coords["lon"][0], mask.coords["lat"][0],mask.coords["lon"][-1], mask.coords["lat"][-1])
bbox

(<xarray.DataArray 'lon' ()>
 array(119.4)
 Coordinates:
     lon      float64 119.4,
 <xarray.DataArray 'lat' ()>
 array(41.7)
 Coordinates:
     lat      float64 41.7,
 <xarray.DataArray 'lon' ()>
 array(130.8)
 Coordinates:
     lon      float64 130.8,
 <xarray.DataArray 'lat' ()>
 array(51.7)
 Coordinates:
     lat      float64 51.7)

In [13]:
data = era5.open(['Total precipitation','10 metre U wind component'], start_time=start_time, end_time=end_time, bbox=bbox, time_chunks=720)
data

/home/zhujianfeng/.conda/envs/stac-test/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/zhujianfeng/.conda/envs/stac-test/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.a

<xarray.Dataset>
Dimensions:  (time: 35041, lon: 115, lat: 101)
Coordinates:
  * lat      (lat) float32 41.7 41.8 41.9 42.0 42.1 ... 51.3 51.4 51.5 51.6 51.7
  * lon      (lon) float32 119.4 119.5 119.6 119.7 ... 130.5 130.6 130.7 130.8
  * time     (time) datetime64[ns] 2018-01-01 2018-01-01T01:00:00 ... 2021-12-31
Data variables:
    tp       (time, lon, lat) float32 dask.array<chunksize=(216, 115, 101), meta=np.ndarray>
    u10      (time, lon, lat) float32 dask.array<chunksize=(216, 115, 101), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-21 12:13:40 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

## 计算平均值

In [7]:
import mean

In [16]:
%%time
m = mean.mean_by_mask(data,'tp',mask)
m

CPU times: user 29.5 s, sys: 20.4 s, total: 49.9 s
Wall time: 21 s


dask.array<truediv, shape=(35041,), dtype=float64, chunksize=(20,), chunktype=numpy.ndarray>

In [14]:
m.compute()

array([6.10001307e-05, 2.41910277e-07, 4.49741404e-07, 6.44585677e-07,
       7.60703091e-07, 8.17023707e-07, 8.49789132e-07, 9.07364589e-07,
       9.35017068e-07, 9.57725598e-07, 9.73677996e-07, 9.80410537e-07,
       9.84841905e-07, 9.89876427e-07, 9.95278481e-07, 9.96665940e-07,
       9.97271662e-07, 9.97867196e-07, 1.00338654e-06, 1.00579270e-06,
       1.01573304e-06, 1.02025529e-06, 1.02784512e-06, 1.03331000e-06,
       8.48700116e-07, 1.68735214e-09, 2.54054261e-09, 3.24214787e-09,
       3.65948356e-09, 4.43482378e-09, 4.80864812e-09, 9.76861620e-09,
       1.64638628e-08, 2.41609004e-08, 3.62291957e-08, 5.09364836e-08,
       6.02528051e-08, 6.85465798e-08, 7.53548234e-08, 8.05163191e-08,
       8.26194958e-08, 8.36293809e-08, 8.54381544e-08, 9.50875869e-08,
       1.06347528e-07, 1.37943954e-07, 1.84381020e-07, 2.53464388e-07,
       3.36142455e-07, 1.17998631e-07, 2.26407068e-07, 3.00525116e-07,
       3.42040250e-07, 3.85835045e-07, 4.51946859e-07, 6.50739348e-07,
      